In [1]:
from utils.dataset import AirDelayDataset

In [2]:
add = AirDelayDataset('../datasets/airline-delay-analysis/')

Initializing Spark...
Spark stand by (Master=local, AppName=0d47713c-a3a2-4f4d-bd83-1ac108fd9e9b)
Load datasets from ../datasets/airline-delay-analysis/


Loading 2019.csv: : 11it [05:23, 29.45s/it]


Merging Frames...
                                                   0
Spark Master                                   local
Spark APP Name  0d47713c-a3a2-4f4d-bd83-1ac108fd9e9b
Data Path        ../datasets/airline-delay-analysis/
Data Count                                  68979001


In [9]:
add.dataframe().select('FL_DATE').sort('FL_DATE', ascending=False).show()

+-------------------+
|            FL_DATE|
+-------------------+
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
|2009-12-31 00:00:00|
+-------------------+
only showing top 20 rows



In [3]:
add_ori_gnv = add.get_ports(['GNV'], 'origin')
add_ori_gnv.dataframe().show()

+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+
|            FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 27|
+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------

In [3]:
add_period = add.get_date_period('2019-01-01', '2019-01-02')
add_period.dataframe().show()

+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+----+
|            FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 27|_c20|
+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+------------

In [9]:
add_cancel = add.get_cancelled()
add_cancel.dataframe().show()

+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+----+
|            FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 27|_c20|
+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+------------

In [10]:
cancel_num = len(add_cancel)
total_num = len(add)

In [7]:
print('cancel={}, total={}, {:.2f}%'.format(cancel_num,
                                            total_num,
                                            cancel_num/total_num*100))

cancel=973209, total=68979001, 1.41%


In [9]:
add.dataframe().schema

StructType([StructField('FL_DATE', TimestampType(), True), StructField('OP_CARRIER', StringType(), True), StructField('OP_CARRIER_FL_NUM', IntegerType(), True), StructField('ORIGIN', StringType(), True), StructField('DEST', StringType(), True), StructField('CRS_DEP_TIME', DoubleType(), True), StructField('DEP_TIME', DoubleType(), True), StructField('DEP_DELAY', DoubleType(), True), StructField('TAXI_OUT', DoubleType(), True), StructField('WHEELS_OFF', DoubleType(), True), StructField('WHEELS_ON', DoubleType(), True), StructField('TAXI_IN', DoubleType(), True), StructField('CRS_ARR_TIME', DoubleType(), True), StructField('ARR_TIME', DoubleType(), True), StructField('ARR_DELAY', DoubleType(), True), StructField('CANCELLED', DoubleType(), True), StructField('CANCELLATION_CODE', StringType(), True), StructField('DIVERTED', DoubleType(), True), StructField('CRS_ELAPSED_TIME', DoubleType(), True), StructField('ACTUAL_ELAPSED_TIME', DoubleType(), True), StructField('AIR_TIME', DoubleType(), T

In [6]:
add_delay = add.get_delay([0, 0xFFFF], 'arrival')
delay_count = add_delay.dataframe().count()

In [11]:
print('delay={}, total={}, {:.2f}%'.format(delay_count,
                                            total_num,
                                            delay_count/total_num*100))

delay=26597222, total=68979001, 38.56%


In [6]:
len(add_period)

18009

In [8]:
add_period.airposrt_data_path = '../datasets/AIRPORTS_INFO.csv'
add_period_airport = add_period.attach_airport_info()

In [9]:
add_period_airport.dataframe().show()

+----+-------------------+----------+-----------------+------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+----+------------------+------------------+-----------+
|DEST|            FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 27|_c20|          DEST_LAT|          DEST_LON|DEST_REGION|
+----+-------------------+----------+-----------------+------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+-----